# Tutorial 1: Annotation of computational models and tools in existing SPARC datasets
SPARC datasets contain measurements, and also tools and computational models to process and interpret the measurements. 

In this tutorial, we show how we can extract these tools and computational models in existing SPARC datasets and store them in their own SDS dataset to ensure they can be FAIR.

We then show how we can annotate these tools and computational models such that they can be used by sparc-assemble. The annotations will be stored with the SDS that was just created.

Tools and models will be extracted from two exemplar datasets. The examples will be used in subsequent tutorials to demonstrate sparc-assemble functionality including automated assembly of workflows.

** data is sourced from https://sparc.science/datasets/262?type=dataset

## Objectives of tutorial:
- Extract tools and models from SDS dataset
- Store tools and models as SDS dataset
- Link the two new SDS datasets back to original dataset

## Install required modules

In [1]:
!pip install json
!pip install numpy
!pip install sparc.client
!pip install sparc_me
!pip install sparc_flow
!pip install sparc_assemble
#!pip install pyeCAP

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Example 1: extracting and annotating with tools

### Step 1: Get dataset ID

First, we must download a dataset from the SPARC portal. Two key pieces of information are needed:
- Dataset ID
- Dataset version ID

These can be found on the dataset's page of the SPARC portal. We will use this [example dataset](https://sparc.science/datasets/262?type=dataset) for this tutorial.


In [2]:
#   Dataset info
datasetId = 262
versionId = 1.1

### Step 2: Download the dataset

We can either use the `sparc.python` [client](https://docs.sparc.science/docs/sparc-python-client) or the `sparc_me` [repository](https://github.com/SPARC-FAIR-Codeathon/sparc-me) to download a SDS dataset. Alternatively, you can manually download the dataset on the SPARC portal. 

In [3]:
# Automatically download dataset from SPARC portal using their ID
use_sparc_me = True

if use_sparc_me:
    from sparc_me import Dataset_Api
    api_tools = Dataset_Api()
    metadata = api_tools.get_metadata_pensieve(datasetId, versionId)
    dataset = api_tools.get_dataset_latest_version_pensieve(datasetId)
    #api_tools.download_dataset(datasetId)

else:
    from sparc.client import SparcClient
    client = SparcClient(connect=False, config_file='config.ini')
    client.pennsieve.get(f'https://api.pennsieve.io/discover/datasets/{datasetId}/versions/{versionId}/files/browse')

True


Note that Line 9 of the above code section was commented out, as the selected dataset is >29.63 GB!! 

For this tutorial's purpose, we downloaded the target subject data directly from the SPARC portal, processed the data and saved the outputs in a separate folder called *dataset*. 

With `sparc_me`, we can verify that the correct dataset has been downloaded by checking the name and id of dataset

In [4]:
dataset["name"], dataset["id"]

('Electrode design characterization for electrophysiology from swine peripheral nervous system',
 262)

### Step 3: Extract tools and store in SDS using sparc-flow

Here, we demonstrate the use of two tools applied on this dataset. The first tool extracts data from a dataset and outputs a file containing the measurements for the independent variable of the dataset. This becomes an input to the second tool which outputs a file containing measurements for the dependent variable. 

In this dataset, the dependent variable is the membrane voltage of the evoked compound action potentials stimulated by electrodes on the cervical vagus nerve of pigs.  These measurements were recorded over a certain period of time, which is our independent variable.

### Source code

#### Tool 1

In [8]:
import numpy as np
import os

In [9]:
#   Source code
def extract_independent_variable_sds(datasetId, versionId, outputFile):
    """
    This function extracts an array of measurements of the independent
    variable quantity of a SDS dataset with a known Pensieve dataset ID,
    and saves the array to a .txt file.

    :param datasetId: SPARC dataset ID
    :type datasetId: float
    :param versionId: SPARC dataset version ID
    :type versionId: float
    :param outputFile: Output filename for .txt file
    :type outputFile: str
    """

    # Get dataset metadata from SPARC Portal
    api_tools = Dataset_Api()
    metadata = api_tools.get_metadata_pensieve(datasetId, versionId)

    # Note we can download the dataset from SPARC using the sparc-me API using the line below
    # api_tools.download_dataset(dataset["id"])

    # This is commented out as the dataset is >29.63 GB, we downloaded the target subject
    # data directly from the SPARC portal, processed the data and saved the outputs.

    # Independent variable quantity of dataset is time, and this was extracted from the dataset
    x = [0.000000000000000000e+00,
         8.191999999999999629e-04,
         1.638399999999999926e-03,
         2.457599999999999889e-03,
         3.276799999999999852e-03,
         4.095999999999999815e-03,
         4.915199999999999778e-03,
         5.734399999999999741e-03,
         6.553599999999999703e-03,
         7.372799999999999666e-03,
         8.191999999999999629e-03,
         9.011200000000000460e-03,
         9.830399999999999555e-03,
         1.064960000000000039e-02,
         1.146879999999999948e-02,
         1.228800000000000031e-02,
         1.310719999999999941e-02,
         1.392640000000000024e-02,
         1.474559999999999933e-02,
         1.556480000000000016e-02,
         1.638399999999999926e-02,
         1.720319999999999835e-02,
         1.802240000000000092e-02,
         1.884160000000000001e-02,
         1.966079999999999911e-02,
         2.048000000000000168e-02,
         2.129920000000000077e-02,
         2.211839999999999987e-02,
         2.293759999999999896e-02,
         2.375680000000000153e-02,
         2.457600000000000062e-02,
         2.539519999999999972e-02,
         2.621439999999999881e-02,
         2.703360000000000138e-02,
         2.785280000000000047e-02,
         2.867199999999999957e-02,
         2.949119999999999867e-02,
         3.031040000000000123e-02,
         3.112960000000000033e-02,
         3.194879999999999942e-02,
         3.276799999999999852e-02,
         3.358719999999999761e-02,
         3.440639999999999671e-02,
         3.522560000000000274e-02,
         3.604480000000000184e-02,
         3.686400000000000093e-02,
         3.768320000000000003e-02,
         3.850239999999999913e-02,
         3.932159999999999822e-02]

    #   Create output folder if not created
    outputFolder = "outputs"
    os.makedirs(outputFolder, exist_ok=True)

    #   Store dependent variable measurements as .txt file
    np.savetxt(os.path.join(outputFolder, fr"{outputFile}"),
               np.array(x), delimiter=',')

We extract time measurements from the downloaded dataset, and saves it to a .txt file 

In [10]:
extract_independent_variable_sds(
        datasetId=datasetId, versionId=versionId, outputFile='time.txt')

True


#### Tool 2

In [11]:
#   Source code
def extract_dependent_variable_sds(xFile, outputFile):
    """
    This function extracts an array of the dependent variable measurements from a SDS dataset
    at the specified independent variable measurements from tool_extract_indep_var_sds.py.

    :param xFile: independent variable quantity measurements from dataset extracted from a .txt file
    :type xFile: directory
    :param outputFile: file path for saving dependent variable measurements from dataset
    :type outputFile: string
    """

    #   Load file
    x = np.loadtxt(xFile, delimiter=',')

    get_electrodes = False

    #   Electrodes used to measure membrane voltage in each case
    if get_electrodes:
        electrodes = ["Cuff 1", "Cuff 3", "LIFE 2"]

    #   Dependent variable quantity of dataset is membrane voltage obtained from different electrodes
    #   at the specified time measurements (x). This information was extracted from the dataset.
    y = [[-1.194679728397433872e-05,
          -1.980733829726684289e-05,
          -1.077133905927803561e-03,
          -1.519773320369282514e-04,
           4.358444211166163332e-05,
           1.658965415525065864e-05,
           2.398549602516097313e-06,
          -1.332335275220263179e-06,
           1.629816851936227506e-06,
           4.550925948376061261e-07,
          -2.440652094473894876e-07,
          -1.256780047896774447e-07,
          -1.604059700341167774e-07,
          -1.366082574509440694e-07,
           6.310354307689493841e-08,
           1.457346526672138439e-08,
          -1.628435072394222375e-07,
          -6.812635356888434155e-09,
           5.664900883157117720e-08,
          -5.565218230427167767e-09,
           2.460946088137883220e-07,
           5.694319531931366159e-08,
          -2.056334398814257176e-08,
           1.748808934360601829e-07,
           1.773989087065118717e-07,
          -1.101112467177508824e-07,
          -1.075328597482854415e-07,
           3.386144369441476939e-08,
           1.897932070755456980e-08,
          -2.771581229295496183e-08,
          -4.256350579346231629e-08,
           2.369912349741317857e-07,
           1.700902709291848032e-07,
          -1.165152584466342702e-08,
           2.526520739717943143e-08,
           7.121820226943954045e-08,
           7.056591199310062196e-09,
          -5.769651171224500800e-08,
           5.493674015745062083e-08,
          -1.013383430345409753e-07,
           9.970229271565001302e-08,
          -3.204593418913237084e-07,
          -4.325400556337169854e-07,
          -6.636738552043014031e-07,
          -9.158674234832865898e-07,
          -1.763365287119160062e-06,
          -2.739152133168838517e-06,
          -4.476083953312462761e-06,
          -7.542537164392824152e-06],
         [-8.214201655775424599e-06,
          -1.372845595816498555e-05,
          -7.999020217967099580e-04,
          -1.352363618910832938e-04,
           3.092872602419839500e-05,
           2.011895437284948941e-05,
           9.557211317360985987e-06,
          -3.820839410285373378e-06,
          -6.232610444666107165e-07,
           1.988794676647719403e-06,
           1.798649238333572180e-07,
          -6.153998549570629156e-08,
          -2.280486447432810844e-08,
           1.406398600829660458e-07,
           2.274509841389832947e-07,
           2.300306274820511094e-08,
          -1.572653510349146469e-08,
          -3.974936120421572610e-09,
          -1.433040571137541349e-07,
          -6.573999965324418493e-08,
          -2.177186888257862627e-07,
           8.583061039754071028e-08,
           3.093779432661249251e-07,
          -2.944427710333310390e-08,
          -3.330317444469355433e-08,
          -6.162516141635240061e-08,
          -1.202485970381447797e-07,
          -3.076161062497104022e-08,
           2.313064439128372287e-10,
           2.292993146737572915e-08,
          -1.411246583126800866e-07,
          -2.647507798588564735e-08,
           3.457814694547747936e-08,
           3.535925893357380257e-08,
          -3.418105614165993230e-08,
          -1.583688933718167217e-07,
           1.012797271590110401e-07,
           2.562090584582817896e-07,
          -2.444895666644071396e-08,
          -4.979131491438313074e-08,
          -2.076604778027471311e-07,
          -1.452476801605392218e-07,
          -1.909545607780813125e-07,
          -4.140272436638787089e-07,
          -5.639934119954284032e-07,
          -1.227500442716898632e-06,
          -1.947706456334992766e-06,
          -3.273951913389777314e-06,
          -5.192352644414561846e-06],
         [-1.462083138277712432e-06,
          -2.622262173064377229e-06,
          -7.534344594908582143e-04,
           4.759606952424768331e-05,
           9.223688791069610846e-06,
           1.677105825408498600e-05,
           2.933277337991895692e-06,
           8.110282750876216604e-07,
          -1.425914101175619564e-06,
          -1.663120625907863012e-06,
          -9.908095184916553125e-08,
           2.624454673115753441e-07,
          -1.624587473395667151e-07,
           3.338108641988796370e-08,
           4.483677957255745166e-08,
          -2.272849655834033843e-08,
           6.738530411162273816e-08,
          -1.318730119721173922e-07,
           2.174619084598256286e-08,
          -2.635504384574270156e-08,
          -2.126007975966536514e-07,
          -1.018217795072811392e-07,
          -1.916260472310982058e-08,
          -6.106306355050666979e-08,
          -9.495394166931023019e-09,
          -3.854787158901162777e-09,
          -1.056476775362339673e-07,
           4.154432204662217048e-08,
           5.605603793178550658e-08,
          -7.432304252075775190e-08,
          -8.398636677852381007e-08,
          -1.854658209056216439e-07,
          -8.286421105874349958e-08,
          -1.424359400919484305e-07,
          -2.010722883843532184e-07,
           7.429247114041876855e-08,
           6.948458981206337186e-08,
          -1.446947117729709508e-07,
          -5.107443403085964430e-08,
           4.225381293752249147e-10,
           6.443742709884127751e-08,
           4.906226138658068876e-08,
          -9.440736302779392982e-08,
          -2.710204727223795743e-07,
           1.652564732815241083e-09,
          -8.563504499159779379e-08,
          -4.766724708442070395e-07,
          -7.328525557563466932e-07,
          -8.203079358028723468e-07]]

    #   Create output folder if not created
    outputFolder = "outputs"
    os.makedirs(outputFolder, exist_ok=True)

    #   Store dependent variable measurements as .txt file
    np.savetxt(os.path.join(outputFolder, fr"{outputFile}"),
               np.array(y).T, delimiter=',')

This tool takes in the time measurements and extracts the membrane voltage measurements at the provided time points. We then save the membrane voltage measurements to a .txt file. 

In [13]:
extract_dependent_variable_sds(xFile='outputs/time.txt', outputFile='voltage.txt')

### Tool description in CWL

To ensure these tool can be executed reproducibly across different operating systems and environments, we create a CWL file that describes the tools in a standardised manner. 

We can create CWL files by saving the tools' inputs, outputs and descriptions in a json file, which is then used to create a cwl file along the SDS dataset structure. 

In [ ]:
# Tool description in CWL (show what the manually made looks like, where it is)
# Option 1: Hard-code the cwl file
# Option 2: Store the tools' inputs, outputs and descriptions in json file, then use to create a cwl file along the SDS dataset structure

In [14]:
import json

In [15]:
# Create folder for storing .json methods
os.makedirs('methods/', exist_ok=True)

In [68]:
# Save both methods as .json files
tool1_method = {
    "Operation": "tool_extract_indep_var_sds",
    "comment": "gets SDS dataset, retrieves and saves independent variable measurements",
    "Input": {
        "datasetId": {
            "type": "float"
        },
        "versionId": {
         "type": "float"
        },
        "outputFile": {
            "type": "string"
        }
    },
    "Output": {
        "xFile": {
            "type": "File"
        }
    },
    "cwl": "tool_extract_indep_var_sds.cwl"
}

with open(f'methods/{tool1_method["Operation"]}.json', 'w') as f:
    json.dump(tool1_method, f, indent=4)

In [64]:
tool2_method = {
    "Operation": "tool_extract_dep_var_sds",
    "comment": "retrieves and saves dependent variable quantities measured at the corresponding independent variable measurements",
    "Input": {
        "xFile": {
            "type": "File"
        },
        "outputFile": {
            "type": "File"
        }
    },
    "Output": {
        "yFile": {
            "type": "File"
        }
    },
    "cwl": "tool_extract_dep_var_sds.cwl"
}

with open(f'methods/{tool2_method["Operation"]}.json', 'w') as f:
    json.dump(tool2_method, f, indent=4)

In this .json file, note the dictionary key: `"cwl"`. This key stores the cwl file location for the python tool. We can use the tools developed in `sparc_flow` to create a .cwl file and generate an SDS dataset for each tool.

In [ ]:
try:
    import sparc_flow 
except Exception:
    print("Ideally, please run on linux. Sparc_flow does not yet fully support Windows")
    print("If you are a Windows user: please go to the file showing the error message in sparc_flow and comment out 'import pwd'.")

In [85]:
# Save tool 1 in SDS with sparc-flow (check if works, if not, go back to cwl files from my previous commit)
new_tool = sparc_flow.Tool()
new_tool.set_tool_name(tool1_method["Operation"])
new_tool.set_tool_dir(".")
new_tool.set_command(["python", tool1_method["Operation"]])
new_tool.set_input_type(tool1_method["Input"]["datasetId"]["type"])
new_tool.set_input_type(tool1_method["Input"]["versionId"]["type"])
new_tool.set_input_type(tool1_method["Input"]["outputFile"]["type"])
new_tool.set_output_type(tool1_method["Output"]["xFile"]["type"])
new_tool.set_output_path("./outputs/test.txt")
new_tool.generate_description() 
new_tool.create_sds("../resources/tutorial_1_resources/tools1_dataset",
                    ".")

Error: [('.\\dataset\\primary\\sub-1\\perf-sub-1-auricular-stroking\\ECAP1-211013-080914_ECAP1-211013-115925_EMGG_Ch1.sev', '../resources/tutorial_1_resources/tools1_dataset/primary/tools\\dataset\\primary\\sub-1\\perf-sub-1-auricular-stroking\\ECAP1-211013-080914_ECAP1-211013-115925_EMGG_Ch1.sev', "[Errno 2] No such file or directory: '../resources/tutorial_1_resources/tools1_dataset/primary/tools\\\\dataset\\\\primary\\\\sub-1\\\\perf-sub-1-auricular-stroking\\\\ECAP1-211013-080914_ECAP1-211013-115925_EMGG_Ch1.sev'"), ('.\\dataset\\primary\\sub-1\\perf-sub-1-auricular-stroking\\ECAP1-211013-080914_ECAP1-211013-115925_EMGG_Ch2.sev', '../resources/tutorial_1_resources/tools1_dataset/primary/tools\\dataset\\primary\\sub-1\\perf-sub-1-auricular-stroking\\ECAP1-211013-080914_ECAP1-211013-115925_EMGG_Ch2.sev', "[Errno 2] No such file or directory: '../resources/tutorial_1_resources/tools1_dataset/primary/tools\\\\dataset\\\\primary\\\\sub-1\\\\perf-sub-1-auricular-stroking\\\\ECAP1-211013-

In [87]:
# Save tool 2 in SDS with sparc-flow
new_tool = sparc_flow.Tool()
new_tool.set_tool_name(tool2_method["Operation"])
new_tool.set_tool_dir(".")
new_tool.set_command(["python", tool2_method["Operation"]])
new_tool.set_input_type(tool2_method["Input"]["xFile"]["type"])
new_tool.set_input_type(tool2_method["Input"]["outputFile"]["type"])
new_tool.set_output_type(tool2_method["Output"]["yFile"]["type"])
new_tool.set_output_path("./outputs/test.txt")
new_tool.generate_description() 
new_tool.create_sds("../resources/tutorial_1_resources/tools2_dataset",
                    ".")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../resources/tutorial_1_resources/tools2_dataset/primary/tools'

## Example 2: Preparing models (already in SDS)

We can apply the approach above to annotate models from a SPARC dataset, such as this one: https://sparc.science/datasets/135.

The data is related to a mathematical model of the spontaneous electrical activity of a human sinoatrial node pacemaker cell. The dataset provides a `simulation.json` file describing the inputs and outputs of the model and references to a CellML model. 

You can find `simulation.json` via this path relative to the tutorial: `../../datasets/Pennsieve-dataset-135-version-7/files/primary/simulation.json`


One thing to note is that the model is already stored in an SDS datastructure and the `.json` file contains the annotations needed for it's use in `sparc-assembly`. 

Therefore, for this example, we only need to download the dataset. Subsequent tutorials will show how the model can be used in workflow assembly. 

### Step 1: Get dataset ID

In [89]:
# Dataset ID and version ID on SPARC portal
modeldatasetId = 135
modelversionId = 7.0

### Step 2: Download dataset

In [90]:
model_metadata = api_tools.get_metadata_pensieve(modeldatasetId, modelversionId)
model_dataset = api_tools.get_dataset_latest_version_pensieve(modeldatasetId)

True


In [91]:
model_dataset

{'id': 135,
 'sourceDatasetId': 879,
 'name': 'Computational analysis of the human sinus node action potential - Model development and effects of mutations',
 'description': 'CellML version of the Fabbri et al. 2017 mathematical model of the spontaneous electrical activity of a human sinoatrial node (SAN) pacemaker cell',
 'ownerId': 613,
 'ownerFirstName': 'Alan',
 'ownerLastName': 'Garny',
 'ownerOrcid': '0000-0001-7606-5888',
 'organizationName': 'SPARC Consortium',
 'organizationId': 367,
 'license': 'Creative Commons Attribution',
 'tags': ['ion channels',
  'cardiovascular models',
  'sinoatrial node',
  'action potentials',
  'cardiac electrophysiology',
  'cardiomyocyte',
  'human',
  'computer simulation',
  'heart rate'],
 'version': 7,
 'revision': None,
 'size': 167279,
 'modelCount': [{'modelName': 'researcher', 'count': 2},
  {'modelName': 'subject', 'count': 0},
  {'modelName': 'sample', 'count': 0},
  {'modelName': 'term', 'count': 0},
  {'modelName': 'award', 'count': 